In [129]:
# Libraries for downloading data from FTP
import shutil
import urllib.request as req
from contextlib import closing

# Library for uploading data to S3
import boto3

# Libraries for handling data
import rasterio as rio
import pandas as pd
import numpy as np

# Libraries for various helper functions
from datetime import datetime
import os
import threading
import sys

cli.005 https://nsidc.org/data/docs/noaa/g02135_seaice_index/#text_summary
SRC: ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/monthly/geotiff/01_Jan (etc)
files look like: N_197901_concentration_v2.1.tif, N_197901_extent_v2.1.tif
from Jan 1979
File type: tif
North Polar Projection

cli.006 https://nsidc.org/data/docs/noaa/g02135_seaice_index/#text_summary
SRC: ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/south/monthly/geotiff/01_Jan (etc)
files look like: S_197901_concentration_v2.1.tif
from Jan 1979
File type: tif
Antarctic Projection

In [ ]:
## Need to know - how often do they update?

# This could run every day... there won't necessarily be anything to add
# In that case perhaps it should return an error to some console
# It would be nice for RW staff to see what has been updated
# And what updates were attempted but found nothing new


In [116]:
# Directions for how to read contents of an FTP directory
# https://stackoverflow.com/questions/2289768/python-and-urllib
remote_path_north = "ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/monthly/geotiff/"
remote_path_south = "ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/south/monthly/geotiff/"
file = req.urlopen(remote_path_north).read().splitlines()
file

[b'drwxrwxr-x   14 500        9474               14 Apr 21 12:04 .',
 b'drwxrwxr-x    6 500        9474                6 Apr 21 12:04 ..',
 b'drwxr-sr-x    2 500        9474               78 Sep  1 08:20 01_Jan',
 b'drwxr-sr-x    2 500        9474               80 Sep  1 08:20 02_Feb',
 b'drwxrwxr-x    2 500        9474               80 Sep  1 08:20 03_Mar',
 b'drwxr-sr-x    2 500        9474               80 Sep  1 08:20 04_Apr',
 b'drwxr-sr-x    2 500        9474               80 Sep  1 08:20 05_May',
 b'drwxr-sr-x    2 500        9474               80 Sep  1 08:20 06_Jun',
 b'drwxr-sr-x    2 500        9474               80 Sep  1 08:20 07_Jul',
 b'drwxr-sr-x    2 500        9474               80 Sep  1 08:20 08_Aug',
 b'drwxr-sr-x    2 500        9474               78 Sep  1 08:21 09_Sep',
 b'drwxr-sr-x    2 500        9474               78 Sep  1 08:21 10_Oct',
 b'drwxr-sr-x    2 500        9474               80 Sep  1 08:21 11_Nov',
 b'drwxr-sr-x    2 500        9474             

In [117]:
def format_month(mon):
    if mon < 10:
        return("0" + str(mon))
    else:
        return(str(mon))

now = datetime.now()
year = str(now.year)
month = format_month(now.month-1)

def create_most_recent_file(year, month, north_or_south="N"):
    # file_type in [extent, concentration]
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
              "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    return("{}_{}/{}_{}{}_extent_v2.1.tif".format(month,months[int(month)-1],north_or_south, year, month))

arctic_file = create_most_recent_file(year, month, "N")
antarctic_file = create_most_recent_file(year, month, "S")

In [118]:
antarctic_file

'08_Aug/S_201708_extent_v2.1.tif'

In [119]:
ftp_arctic = remote_path_north+arctic_file
ftp_antarctic = remote_path_south+antarctic_file
print(ftp_arctic)
print(ftp_antarctic)

local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/Polar_Sea_Ice/"
# Use 7: to not include the month's folder in the local file name
arctic_file_name = arctic_file[7:]
antarctic_file_name = antarctic_file[7:]

local_arctic = local_folder+arctic_file_name
local_antarctic = local_folder+antarctic_file_name

local_arctic_edit = local_arctic[:-4] + "_edit.tif"
local_antarctic_edit = local_antarctic[:-4] + "_edit.tif"

with closing(req.urlopen(ftp_arctic)) as r:
    with open(local_arctic, 'wb') as f:
        shutil.copyfileobj(r, f)
         
with closing(req.urlopen(ftp_antarctic)) as r:
    with open(local_antarctic, 'wb') as f:
        shutil.copyfileobj(r, f)

ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/monthly/geotiff/08_Aug/N_201708_extent_v2.1.tif
ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/south/monthly/geotiff/08_Aug/S_201708_extent_v2.1.tif


In [131]:
with rio.open(local_arctic) as src:
    pro = src.profile
    print(pro)
    data = src.read()
    
    pro.update(
        compress = "lzw",
        crs = "EPSG:4326"
    )
    
    with rio.open(local_arctic_edit, 'w', **pro) as dst:
        dst.write(data)
        
        
with rio.open(local_antarctic) as src:
    pro = src.profile
    print(pro)
    data = src.read()
    
    pro.update(
        compress = "lzw",
        crs = "EPSG:4326"
    )
    
    with rio.open(local_antarctic_edit, 'w', **pro) as dst:
        dst.write(data)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 304, 'height': 448, 'count': 1, 'crs': CRS({'init': 'epsg:3411'}), 'transform': (-3850000.0, 25000.0, 0.0, 5850000.0, 0.0, -25000.0), 'affine': Affine(25000.0, 0.0, -3850000.0,
       0.0, -25000.0, 5850000.0), 'tiled': False, 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 316, 'height': 332, 'count': 1, 'crs': CRS({'init': 'epsg:3412'}), 'transform': (-3950000.0, 25000.0, 0.0, 4350000.0, 0.0, -25000.0), 'affine': Affine(25000.0, 0.0, -3950000.0,
       0.0, -25000.0, 4350000.0), 'tiled': False, 'interleave': 'band'}


/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


In [121]:
with rio.open(local_arctic_edit) as src:
    pro = src.profile
    print(pro)
    
with rio.open(local_antarctic_edit) as src:
    pro = src.profile
    print(pro)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 304, 'height': 448, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-3850000.0, 25000.0, 0.0, 5850000.0, 0.0, -25000.0), 'affine': Affine(25000.0, 0.0, -3850000.0,
       0.0, -25000.0, 5850000.0), 'compress': 'lzw', 'interleave': 'band', 'tiled': False}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 316, 'height': 332, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-3950000.0, 25000.0, 0.0, 4350000.0, 0.0, -25000.0), 'affine': Affine(25000.0, 0.0, -3950000.0,
       0.0, -25000.0, 4350000.0), 'compress': 'lzw', 'interleave': 'band', 'tiled': False}


In [127]:
### NOTE = THIS WILL KEEP REPLACING THE FILE WITH THE NEWEST VERSION

# S3 destinations
s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/"

s3_file_antarctic = "cli_005_antarctic_sea_ice_extent.tif"
s3_key_antarctic_orig = s3_folder + s3_file_antarctic
s3_key_antarctic_edit = s3_key_antarctic_orig[:-4] + "_edit.tif"

s3_file_arctic = "cli_006_arctic_sea_ice_extent.tif"
s3_key_arctic_orig = s3_folder + s3_file_arctic
s3_key_arctic_edit = s3_key_arctic_orig[:-4] + "_edit.tif"

# S3 services
s3_download = boto3.resource("s3")
s3_upload = boto3.client("s3")

# Helper function to view upload progress
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [130]:
# Originals
s3_upload.upload_file(local_arctic, s3_bucket, s3_key_arctic_orig,
                         Callback=ProgressPercentage(local_arctic))

s3_upload.upload_file(local_antarctic, s3_bucket, s3_key_antarctic_orig,
                         Callback=ProgressPercentage(local_antarctic))

# Edits
s3_upload.upload_file(local_arctic_edit, s3_bucket, s3_key_arctic_edit,
                         Callback=ProgressPercentage(local_arctic_edit))

s3_upload.upload_file(local_antarctic_edit, s3_bucket, s3_key_antarctic_edit,
                         Callback=ProgressPercentage(local_antarctic_edit))

/Users/nathansuberi/Desktop/RW_Data/Rasters/Polar_Sea_Ice/S_201708_extent_v2.1_edit.tif  5005 / 5005.0  (100.00%)